# **Thesis v2**

In [ ]:
import os
import sys
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
import re
import time
import datetime
import nltk
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.layers import Input, Dense, GRU, Embedding, CuDNNGRU, LSTM
from tensorflow.keras.models import Model
from tensorflow.python.keras.callbacks import ModelCheckpoint
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.python.training import training
from tensorflow.python.client import device_lib
from google.colab import drive
from google.colab import files
from tqdm import tqdm

Using TensorFlow backend.


In [ ]:
# !pip install tensorflow==2.4.1
# !pip install keras==2.3.1

In [ ]:
try:
  drive.mount('/content/drive')
  os.chdir("drive/My Drive/GoogleColab/")
  print('Dosya dizini belirtilen konuma ayarlandı.')
except:
  print('Dosya dizini belirtilen konumda...')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Dosya dizini belirtilen konuma ayarlandı.


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Not connected to a GPU


In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 37.8 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
try:
  %load_ext tensorboard
  print("TensorBoard eklentisi kuruldu.")
finally:
  %rm -rf ./logs/
  print("Önceki çalıştırmalardan elde edilen günlükler temizlendi.")

TensorBoard eklentisi kuruldu.
Önceki çalıştırmalardan elde edilen günlükler temizlendi.


In [ ]:
if tf.executing_eagerly():
  print("İstekli çalışma modu çalışıyor.")
else:
  tf.compat.v1.enable_eager_execution()
  print("İstekli çalışma moduna geçildi.")

İstekli çalışma modu çalışıyor.


In [ ]:
print("TensorFlow version: {}".format(tf.__version__))
print("Eager execution: {}".format(tf.executing_eagerly()))

TensorFlow version: 2.4.1
Eager execution: True


### **Verileri Yükleme**

In [ ]:
document = pd.read_csv('preparedSuDerDataset.csv')
document.drop(['Unnamed: 0'], axis=1, inplace=True)

news_texts = document['News']
title_texts = document['Title']

series_news_texts = pd.Series(news_texts)
series_title_texts = pd.Series(title_texts)

**Dataset Train ve Validation Olarak Bölme**




---



In [ ]:
news_train, news_validation, title_train, title_validation = train_test_split(series_news_texts, series_title_texts, test_size=0.1)

**Metinleri tokenlere(jeton, vektör) Dönüştürme**

---



In [ ]:
# '<' ve '>' varsayılan belirteçlerden kaldırılamaz. Çünkü dekoder için hedef derlemimizi bu işaretler arasına aldık
# Gerekli olan filtreleme işlemini news_preprocess ve title_preprocess fonksionlarıyla yaptık
filters = '!"$%()*+,./:;=?@[\\]^_{|}~\t\n'

In [ ]:
#news_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters=filters)
#title_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters=filters)
news_tokenizer = keras.preprocessing.text.Tokenizer(filters=filters, oov_token='<unk>', num_words=2)
title_tokenizer = keras.preprocessing.text.Tokenizer(filters=filters, oov_token='<unk>', num_words=2)

In [ ]:
print("Haber metinlerinin vektör sözlüğü oluşturuluyor...")
news_tokenizer.fit_on_texts(news_train)
print("Haber metinlerinin vektör sözlüğü oluşturuldu.")

print("Haber başlıklarının vektör sözlüğü oluşturuluyor...")
news_tokenizer.fit_on_texts(title_train)
# title_tokenizer.fit_on_texts(title_train)
print("Haber başlıklarının vektör sözlüğü oluşturuldu.")
# --------------------------------------------------------------------
# print("Haber metinlerinin vektör sözlüğü oluşturuluyor...")
# news_tokenizer.fit_on_texts(series_news_texts)
# print("Haber metinlerinin vektör sözlüğü oluşturuldu.")

# print("Haber başlıklarının vektör sözlüğü oluşturuluyor...")
# news_tokenizer.fit_on_texts(series_title_texts)
# print("Haber başlıklarının vektör sözlüğü oluşturuldu.")

Haber metinlerinin vektör sözlüğü oluşturuluyor...
Haber metinlerinin vektör sözlüğü oluşturuldu.
Haber başlıklarının vektör sözlüğü oluşturuluyor...
Haber başlıklarının vektör sözlüğü oluşturuldu.


In [ ]:
print("Haber metinleri vektörleştiriliyor...")
inputs = news_tokenizer.texts_to_sequences(news_train)
print("Haber metinleri vektörleştirildi.")

print("Haber başlıkları vektörleştiriliyor...")
# targets = title_tokenizer.texts_to_sequences(title_train)
targets = news_tokenizer.texts_to_sequences(title_train)
print("Haber başlıkları vektörleştirildi.")

Haber metinleri vektörleştiriliyor...
Haber metinleri vektörleştirildi.
Haber başlıkları vektörleştiriliyor...
Haber başlıkları vektörleştirildi.


In [ ]:
encoder_vocab_size = len(news_tokenizer.word_index) + 1 
decoder_vocab_size = len(news_tokenizer.word_index) + 1
# decoder_vocab_size = len(title_tokenizer.word_index) + 1

print('Encoder sözlük sayısı: {}\nDecoder sözlük sayısı: {}'.format(encoder_vocab_size, decoder_vocab_size))

Encoder sözlük sayısı: 806744
Decoder sözlük sayısı: 806744


**Haber ve başlık metinlerinin ayrı ayrı uzunluklarını tanımlama**

In [ ]:
# news_train_lengths = pd.Series([len(x.split()) for x in news_train])
# title_train_lengths = pd.Series([len(x.split()) for x in title_train])

# news_validation_lengths = pd.Series([len(x.split()) for x in news_validation])
# title_validation_lengths = pd.Series([len(x.split()) for x in title_validation])

In [ ]:
encoder_maxlen = 352
decoder_maxlen = 12

In [ ]:
inputs = tf.keras.preprocessing.sequence.pad_sequences(
    inputs,
    maxlen=encoder_maxlen,
    padding='post',
    truncating='post')

targets = tf.keras.preprocessing.sequence.pad_sequences(
    targets,
    maxlen=decoder_maxlen,
    padding='post',
    truncating='post')

In [ ]:
# tf.cast almış olduğu x vektör dizisini (bu kod bloğunda inputs ve targets) tensör olarak döndürür. Tensörler numpy() dizileri gibi düşünülebilir
encoder_input_data = inputs
decoder_input_data = targets[:, :-1]
decoder_output_data = targets[:, 1:]

inputs = tf.cast(encoder_input_data, dtype=tf.int32)
decoder_input_data = tf.cast(decoder_input_data, dtype=tf.int32)
decoder_output_data = tf.cast(decoder_output_data, dtype=tf.int32)

In [ ]:
BATCH_SIZE = 128
EMBEDDING_SIZE = 256 # Sözlük boyutu (dimension)
STATE_SIZE = 256 # Katmanlar arası nöron sayısı

In [ ]:
# encoder_embedding = Embedding(input_dim = encoder_vocab_size, output_dim = EMBEDDING_SIZE, name = 'encoder_embedding')

In [ ]:
# encoder_gru1 = LSTM(STATE_SIZE, name='encoder_gru1', return_sequences = True)
# encoder_gru2 = LSTM(STATE_SIZE, name='encoder_gru2', return_sequences = True)
# encoder_gru3 = LSTM(STATE_SIZE, name='encoder_gru3', return_sequences = False)

In [ ]:
# encoder_input = Input(shape=(None,), name='encoder_input')

In [ ]:
# def connect_encoder():
#     net = encoder_input
#     net = encoder_embedding(net)
    
#     net = encoder_gru1(net)
#     net = encoder_gru2(net)
#     net = encoder_gru3(net)
    
#     encoder_output = net
    
#     return encoder_output

In [ ]:
# Encoder
# encoder_inputs = Input(shape=(encoder_maxlen, ))
encoder_inputs = Input(shape=(None, ))

# Embedding layer
encoder_embedding = Embedding(encoder_vocab_size, EMBEDDING_SIZE, name="EncoderEmbedding") # arg* ---> trainable=True
 
# Encoder LSTM 1
encoder_lstm1 = LSTM(STATE_SIZE, return_sequences=True,return_state=True, dropout=0.4,recurrent_dropout=0.4, name='EncoderLSTM-1')
# Encoder LSTM 2
encoder_lstm2 = LSTM(STATE_SIZE, return_sequences=True,return_state=True, dropout=0.4,recurrent_dropout=0.4, name="EncoderLSTM-2")
# Encoder LSTM 3
encoder_lstm3 = LSTM(STATE_SIZE, return_state=True, return_sequences=True, dropout=0.4,recurrent_dropout=0.4, name="EncoderLSTM-3")

def connect_encoder():

    net = encoder_embedding(encoder_inputs)
    
    (encoder_outputs1, state_h1, state_c1) = encoder_lstm1(net)
    (encoder_outputs2, state_h2, state_c2) = encoder_lstm2(encoder_outputs1)
    (encoder_outputs, state_h, state_c) = encoder_lstm3(encoder_outputs2)

    return encoder_outputs, state_h, state_c

In [ ]:
encoder_outputs, state_h, state_c = connect_encoder()
initial_state = [state_h, state_c]

In [ ]:
decoder_initial_state = Input(shape=(STATE_SIZE,), name='decoder_initial_size')

In [ ]:
# decoder_input = Input(shape=(None, ), name='decoder_input')

In [ ]:
# decoder_embedding = Embedding(input_dim = encoder_vocab_size, output_dim = EMBEDDING_SIZE, name = 'encoder_embedding')

In [ ]:
# decoder_gru1 = LSTM(STATE_SIZE, name='decoder_gru1', return_sequences=True)
# decoder_gru2 = LSTM(STATE_SIZE, name='decoder_gru2', return_sequences=True)
# decoder_gru3 = LSTM(STATE_SIZE, name='decoder_gru3', return_sequences=True)

In [ ]:
# decoder_dense = Dense(decoder_vocab_size, activation='linear', name='decoder_output')

In [ ]:
# def connect_decoder(initial_state):
#     net = decoder_input

#     net = decoder_embedding(net)

#     net, hidden_h = decoder_gru1(net, initial_state = initial_state)
#     net, hidden_h,hidden_c = decoder_gru2(net, initial_state = initial_state)
#     net, hidden_h,hidden_c = decoder_gru3(net, initial_state = initial_state)

#     # net = decoder_gru1(net, initial_state=initial_state)
#     # net = decoder_gru2(net, initial_state=initial_state)
#     # net = decoder_gru3(net, initial_state=initial_state)
    
#     decoder_output, hidden_h,hidden_c = decoder_dense(net)
    
#     return decoder_output

In [ ]:
# def connect_decoder(initial_state):

#     net = decoder_input
#     net = decoder_embedding(net)
    
#     (decoder_outputs1, decoder_fwd_state, decoder_back_state) = decoder_lstm1(dec_emb, initial_state=[state_h, state_c])
#     net = decoder_gru1(net, initial_state=initial_state)
#     net = decoder_gru2(net, initial_state=initial_state)
#     net = decoder_gru3(net, initial_state=initial_state)
    
#     decoder_output = decoder_dense(net)
    
#     return decoder_output

In [ ]:
# Set up the decoder, using encoder_states as the initial state
decoder_inputs = Input(shape=(None, ))

# Embedding layer
decoder_embedding = Embedding(decoder_vocab_size, EMBEDDING_SIZE, trainable=True, name="DecoderEmbedding")

# Decoder LSTM
decoder_lstm1 = LSTM(STATE_SIZE, return_sequences=True, return_state=True, dropout=0.4, recurrent_dropout=0.2, name="DecoderLSTM-1")
# Decoder LSTM 2
decoder_lstm2 = LSTM(STATE_SIZE, return_sequences=True,return_state=True, dropout=0.4,recurrent_dropout=0.4, name="DecoderLSTM-2")
# Decoder LSTM 3
decoder_lstm3 = LSTM(STATE_SIZE, return_state=True,return_sequences=True, dropout=0.4,recurrent_dropout=0.4, name="DecoderLSTM-3")
# Dense layer
decoder_dense = TimeDistributed(Dense(decoder_vocab_size, activation='softmax'), name="DecoderDense")

def connect_decoder(initial_state):

    dec_emb = decoder_embedding(decoder_inputs)
    
    (decoder_outputs, decoder_fwd_state, decoder_back_state) = decoder_lstm1(dec_emb, initial_state=initial_state)
    
    decoder_outputs = decoder_dense(decoder_outputs)
    
    return decoder_outputs

In [ ]:
decoder_outputs = connect_decoder(initial_state)

In [ ]:
model_train = Model(inputs=[encoder_inputs, decoder_inputs], outputs=decoder_outputs)

In [ ]:
model_encoder = Model(inputs=[encoder_inputs], outputs=[encoder_outputs])

In [ ]:
# decoder_output = connect_decoder(initial_state=decoder_initial_state)
model_decoder = Model(inputs=[decoder_inputs, encoder_inputs], outputs=[decoder_outputs])

In [ ]:
def sparse_cross_entropy(y_true, y_pred):
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred)
    loss_mean = tf.reduce_mean(loss)
    return loss_mean

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

In [ ]:
model_train.compile(optimizer=optimizer, loss=sparse_cross_entropy, metrics=['accuracy'], run_eagerly=True)

In [ ]:
# checkpoint_path = "/checkpoints/rev2/lstm/metrics/train/keras_embedding.ckpt-0.data-00000-of-00001"
# checkpoint = ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True)

In [ ]:
try:
    model_train.load_weights(checkpoint_path)
    print('Eğitim ağırlıkları yüklendi...\n')
except Exception as error:
    print('Checkpoint yüklenirken hata oluştu. Eğitime başlanıyor...\n')
    print(error)

Checkpoint yüklenirken hata oluştu. Eğitime başlanıyor...

Unsuccessful TensorSliceReader constructor: Failed to find any matching files for /checkpoints/rev2/lstm/metrics/train/keras_embedding.ckpt-0.data-00000-of-00001


In [ ]:
!ls

In [ ]:
x_data = {'encoder_input':encoder_input_data, 'decoder_input':decoder_input_data}
y_data = {'decoder_output':decoder_output_data}

In [ ]:
callbacks = tf.keras.callbacks.TensorBoard(log_dir='checkpoints/rev2/lstm/metrics', histogram_freq=1, write_graph=True, write_images=False, update_freq='epoch', profile_batch=2, embeddings_freq=1, embeddings_metadata=None)

In [ ]:
model_train.summary()

In [ ]:
model_decoder.summary()

In [ ]:
model_encoder.summary()

In [ ]:
history = model_train.fit(x=[encoder_input_data, decoder_input_data], y=decoder_output_data, batch_size=64, epochs=25, callbacks=[callbacks], validation_split=0.2)

# **Sequential**

In [ ]:
# Sequential

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

latent_dim = STATE_SIZE # state size
embedding_dim = EMBEDDING_SIZE

# Encoder
encoder_inputs = Input(shape=(encoder_maxlen, ))

# Embedding layer
enc_emb = Embedding(encoder_vocab_size, embedding_dim, trainable=True)(encoder_inputs)

# Encoder LSTM 1
encoder_lstm1 = LSTM(latent_dim, return_sequences=True,return_state=True, dropout=0.4,recurrent_dropout=0.4)
(encoder_output1, state_h1, state_c1) = encoder_lstm1(enc_emb)

# Encoder LSTM 2
encoder_lstm2 = LSTM(latent_dim, return_sequences=True,return_state=True, dropout=0.4,recurrent_dropout=0.4)
(encoder_output2, state_h2, state_c2) = encoder_lstm2(encoder_output1)

# Encoder LSTM 3
encoder_lstm3 = LSTM(latent_dim, return_state=True, return_sequences=False, dropout=0.4,recurrent_dropout=0.4)
(encoder_outputs, state_h, state_c) = encoder_lstm3(encoder_output2)

# Set up the decoder, using encoder_states as the initial state
decoder_inputs = Input(shape=(None, ))

# Embedding layer
dec_emb_layer = Embedding(decoder_vocab_size, embedding_dim, trainable=True)
dec_emb = dec_emb_layer(decoder_inputs)

# Decoder LSTM
decoder_lstm1 = LSTM(latent_dim, return_sequences=True, return_state=True, dropout=0.4, recurrent_dropout=0.2)
(decoder_outputs1, decoder_fwd_state, decoder_back_state) = decoder_lstm1(dec_emb, initial_state=[state_h, state_c])

# Decoder LSTM 2
decoder_lstm2 = LSTM(latent_dim, return_sequences=True,return_state=True, dropout=0.4,recurrent_dropout=0.4)
(decoder_output2, state_h3, state_c3) = decoder_lstm2(decoder_outputs1)

# Decoder LSTM 3
decoder_lstm3 = LSTM(latent_dim, return_state=True,return_sequences=True, dropout=0.4,recurrent_dropout=0.4)
(decoder_outputs3, state_h4, state_c4) = decoder_lstm3(decoder_output2)

# Dense layer
decoder_dense = TimeDistributed(Dense(decoder_vocab_size, activation='softmax'))
decoder_outputs = decoder_dense(decoder_outputs3)

# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.summary()

In [ ]:
model.compile(optimizer=optimizer, loss=sparse_cross_entropy, metrics=['accuracy'], run_eagerly=True)

In [ ]:
checkpoint_path = "checkpoints/rev2/gru"
checkpoint = ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True)

In [ ]:
try:
    model_train.load_weights(path_checkpoint)
except Exception as error:
    print('Checkpoint yüklenirken hata oluştu. Eğitime başlanıyor...\n')
    print(error)

Checkpoint yüklenirken hata oluştu. Eğitime başlanıyor...

name 'model_train' is not defined


In [ ]:
x_data = {'encoder_input':encoder_input_data, 'decoder_input':decoder_input_data}
y_data = {'decoder_output':decoder_output_data}

In [ ]:
callbacks = tf.keras.callbacks.TensorBoard(log_dir='checkpoints/rev2/gru/metrics', histogram_freq=1, write_graph=True, write_images=False, update_freq='epoch', profile_batch=2, embeddings_freq=1, embeddings_metadata=None)

In [ ]:
history=model.fit([encoder_input_data,decoder_input_data],decoder_output_data,epochs=50,callbacks=[callbacks],batch_size=8)

In [ ]:
# Visualize the model learning
from matplotlib import pyplot
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

In [ ]:
# Next, let’s build the dictionary to convert the index to word for target and source vocabulary:
reverse_target_word_index=title_tokenizer.index_word
reverse_source_word_index=news_tokenizer.index_word
target_word_index=title_tokenizer.word_index

In [ ]:
# Encode the input sequence to get the feature vector
encoder_model = Model(inputs=encoder_inputs,outputs=[encoder_outputs, state_h, state_c])

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(STATE_SIZE,))
decoder_state_input_c = Input(shape=(STATE_SIZE,))
decoder_hidden_state_input = Input(shape=(encoder_maxlen,STATE_SIZE))

# Get the embeddings of the decoder sequence
dec_emb2= dec_emb_layer(decoder_inputs) 
# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=[decoder_state_input_h, decoder_state_input_c])

# A dense softmax layer to generate prob dist. over the target vocabulary
decoder_outputs2 = decoder_dense(decoder_outputs2) 

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + [decoder_hidden_state_input,decoder_state_input_h, decoder_state_input_c],
    [decoder_outputs2] + [state_h2, state_c2])

In [ ]:
# We are defining a function below which is the implementation of the inference process
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    e_out, e_h, e_c = encoder_model.predict(input_seq)
    
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    
    # Populate the first word of target sequence with the start word.
    target_seq[0, 0] = target_word_index['sostok']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
      
        output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c])

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = reverse_target_word_index[sampled_token_index]
        
        if(sampled_token!='eostok'):
            decoded_sentence += ' '+sampled_token

        # Exit condition: either hit max length or find stop word.
        if (sampled_token == 'eostok'  or len(decoded_sentence.split()) >= (max_summary_len-1)):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update internal states
        e_h, e_c = h, c

    return decoded_sentence

In [ ]:
# Let us define the functions to convert an integer sequence to a word sequence for summary as well as the reviews:
def seq2summary(input_seq):
    newString=''
    for i in input_seq:
        if((i!=0 and i!=target_word_index['sostok']) and i!=target_word_index['eostok']):
            newString=newString+reverse_target_word_index[i]+' '
    return newString

def seq2text(input_seq):
    newString=''
    for i in input_seq:
        if(i!=0):
            newString=newString+reverse_source_word_index[i]+' '
    return newString

In [ ]:
# Run the model over the data to see the results
for i in range(0,100):
    print("Review:",seq2text(news_train[i]))
    print("Original summary:",seq2summary(title_train[i]))
    print("Predicted summary:",decode_sequence(news_train[i].reshape(1,maxlen)))
    print("\n")

# **Compute Score**

### **BLEU**

In [ ]:
!pip install --upgrade bleu
from bleu import list_bleu

In [ ]:
ref = ["""thy nin yolcu sayısı ### milyona ulaştı"""]
hyp = ["""THY yolcu sayısını 9.2 milyona çıkardı"""]

In [ ]:
list_bleu([ref], hyp)



---



In [ ]:
import nltk

In [ ]:
hypothesis = "### terörist etkisiz hale getirildi".split()
reference = title_preprocess("Bitlis'te 6 teröristin öldürüldüğü operasyonda çok sayıda silah mühimmat ele geçirildi").split()
#there may be several references
BLEUscore = nltk.translate.bleu_score.sentence_bleu([reference], hypothesis,auto_reweigh=True)
print(BLEUscore*100)



---



In [ ]:
# two references for one document
from nltk.translate.bleu_score import corpus_bleu
references = [["".split()]]
candidates = ["".split(), "".split()]
score = corpus_bleu(references, candidates)
print(score)

### **ROUGE**

In [ ]:
!pip install 0 0 

In [ ]:
from rouge_metric import PyRouge

By default, sentences are separated by '\n' and tokens are separated by white space in a document. This tokenization process can be further customized. For example,

In [ ]:
# # Template
# from rouge_metric import PyRouge

# # Pre-process and tokenize the summaries as you like
# hypotheses = [
#     ['how are you'.split(), 'i am fine'.split()],                       # document 1: hypothesis
#     ['it is fine today'.split(), 'we won the football game'.split()],   # document 2: hypothesis
# ]
# references = [[
#     ['how do you do'.split(), 'fine thanks'.split()],   # document 1: reference 1
#     ['how old are you'.split(), 'i am three'.split()],  # document 1: reference 2
# ], [
#     ['it is sunny today'.split(), 'let us go for a walk'.split()],  # document 2: reference 1
#     ['it is a terrible day'.split(), 'we lost the game'.split()],   # document 2: reference 2
# ]]

# # Evaluate on tokenized documents
# rouge = PyRouge(rouge_n=(1, 2, 4), rouge_l=True, rouge_w=True,
#                 rouge_w_weight=1.2, rouge_s=True, rouge_su=True, skip_gap=4)
# scores = rouge.evaluate_tokenized(hypotheses, references)
# print(scores)

In [ ]:
# hypotheses = ["kavga ### can alan maganda vurdu".split()]
# references = ["Kavga sebebi sudan faturası ise iki can".split()]

In [ ]:
# print(hypotheses)
# print(references)

In [ ]:
# references = [title.split() for title in title_texts]

In [ ]:
# hypotheses20 = [generate_title(news_preprocess(news)).split() for news in news_texts]

In [ ]:
# hypotheses20 = news_texts.progress_apply(lambda x: generate_title(news_preprocess(str(x))) if i < range(25000))

In [ ]:
hypotheses25 = []
references = []

# tqdm.pandas(desc="Progress")
# for i in tqdm(range(25000,52489)):
#   hypotheses20.append(generate_title(news_preprocess(news_texts[i])).split())
#   references.append(title_texts[i].split())

In [ ]:
dict_rouge = {"Tahmin - 25 Epoch":hypotheses25}

In [ ]:
df = pd.DataFrame(dict_rouge)

In [ ]:
df.to_csv("s25k25e.csv")

In [ ]:
for i in tqdm(range(25000,52489)):
  hypotheses25.append(generate_title(news_preprocess(news_texts[i])))

## **Üretme**

In [ ]:
# token_start = title_tokenizer.word_index[_START_.strip()]
# token_end = title_tokenizer.word_index[_END_.strip()]
_START_ = '<tstart>'
_END_ = '<tend>'
token_start = news_tokenizer.word_index[_START_.strip()]
token_end = news_tokenizer.word_index[_END_.strip()]

In [ ]:
def generate(input_text, true_output_text=None):

  input_tokens = news_tokenizer.texts_to_sequences([input_text])
  input_tokens = np.array(input_tokens)

  initial_state = model_encoder.predict(input_tokens)

  decoder_input_data = np.zeros(shape=(1,decoder_maxlen), dtype=np.int32)

  token_int = _START_
  output_text=' '
  count_tokens=0

  while token_int != token_end and count_tokens < decoder_maxlen:
    decoder_input_data[0, count_tokens] = token_int
    x_data = {'decoder_input':decoder_input_data, 'decoder_initial_size':initial_state}

    decoder_output = model_decoder.predict(x_data)

    token_onehot = decoder_output[0, count_tokens,:]
    token_int = np.argmax(token_onehot)

    sample_word = title_tokenizer.index_word[token_int]
    output_text += ' '+sample_word
    count_tokens+= 1

  print('Girdi:')
  print(input_text)
  print()

  print('Üretililen Başlık:')
  print(output_text)
  print()

  print('Gerçek Başlık:')
  print(true_output_text)
  print()

In [ ]:
index = 39546
news_texts[index], title_texts[index]
n = text_cleaner(news_texts[index])
t = title_cleaner(title_texts[index])
generate(n,true_output_text=t)

In [ ]:
a

['<istart> izmir deki asker tutuklu sanığın yargılandığı gizli bilgi belge bulundurma davasında asker tutuklu sanık tahliye edildi izmir ağır ceza mahkemesi nde görülen davaya tutuklu tutuksuz sanıkların avukatları katıldı mahkeme başkanı duruşma boyunca sürekli izin almadan konuştuğu gerekçesiyle davanın numaralı sanığı bilgin özkaynak salonundan çıkardı mahkeme heyeti tutuklu sanıklardan ercan yabuloğlu meryem erturan muvazzaf askerler atilla kaya bülent kul doğan şahin hasan kızılaslan hüseyin cengiz ibrahim aydın mustafa ufuk kök ihsan kürşat toksoy hakan gürdal burhan kahraman nihat demirhan tahliyesine karar verdi <iend>']

In [ ]:
text = TextCleaner("""Bursa'nın Yıldırım ilçesinde bir vatandaş, sokağa çıkma kısıtlamasını ihlal ederek, şehir içi yolcu taşımacılığında kullanılan minibüste yolculuk etti. Polis ekiplerine "İzin belgem yok ama 'WhatsApp' grubumuz var, olmaz mı? Ben de minibüsçüyüm" diyen şahsa 3 bin 150 TL ceza yazıldı.
Bursa İl Emniyet Müdürlüğü ekipleri, sokağa çıkma kısıtlaması nedeniyle kent genelinde uygulama noktaları oluşturdu. Yıldırım'ın Otosansit Kavşağı'ndaki kontrol noktasında şehir içi yolcu taşımacılığı yapılan otobüs ve minibüsleri denetleyen ekipler, yolcuların izin belgelerini inceledi.""")

In [ ]:
generate(text)

Girdi:
bursa yıldırım ilçesinde vatandaş sokağa çıkma kısıtlamasını ihlal şehir içi yolcu taşımacılığında kullanılan minibüste yolculuk etti polis ekiplerine i̇zin belgem yok whatsapp grubumuz olmaz minibüsçüyüm diyen şahsa 3 bin 150 tl ceza yazıldı bursa i̇l emniyet müdürlüğü ekipleri sokağa çıkma kısıtlaması nedeniyle kent genelinde uygulama noktaları oluşturdu yıldırım ın otosansit kavşağı ndaki kontrol noktasında şehir içi yolcu taşımacılığı yapılan otobüs minibüsleri denetleyen ekipler yolcuların izin belgelerini inceledi

Üretililen Başlık:
  asker diye fırsat telefonu bulunan şüpheli yemek yakalandı <stop>

Gerçek Başlık:
None

